<a href="https://colab.research.google.com/github/davidwbarnes/Playa-Gauge/blob/main/Playa-Gauge_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

David's Colab Report


First section is all about setting up the environement and installing the required packages

In [1]:
!pip install pip
%pip install llama_index
%pip install langchain
%pip install bs4
%pip install openai
!pip install python_dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab_Notebooks/Davids_Reports/


Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Davids_Reports


In [3]:
import logging
import sys
import os
import dotenv
import requests
import datetime
import urllib.request
import ssl



In [4]:
timestamp = datetime.datetime.now()

# Convert the timestamp to a string representation
timestamp_str = timestamp.strftime("%Y-%m-%d")

In [5]:
import dotenv
# My OpenAI Key
dotenv.load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
import ipywidgets as widgets
from IPython.display import display

# Create text widgets for company name and company URL
company_name_widget = widgets.Text(description='Company Name:')
company_url_widget = widgets.Text(description='Company URL:')

# Display the widgets
display(company_name_widget)
display(company_url_widget)



Text(value='', description='Company Name:')

Text(value='', description='Company URL:')

In [12]:
# Wait for user input
company_name = company_name_widget.value
company_url = company_url_widget.value

# Print the values
print("Company Name:", company_name)
print("Company URL:", company_url)


Company Name: Ed App
Company URL: https://www.edapp.com/


## This accesses the website and locates the sitemap.xml

In [13]:
import datetime
import requests

# Provide the company name and their domain name. This section will navigate to the robots.txt and load the sitemap

# Get the current timestamp
import datetime
timestamp = datetime.datetime.now()

# Convert the timestamp to a string representation
timestamp_str = timestamp.strftime("%Y-%m-%d")


def fetch_robots_txt(company_url):
    # Append "/robots.txt" to the provided URL
    robots_url = company_url.rstrip("/") + "/robots.txt"

    # Send a GET request to retrieve the content of the robots.txt file
    response = requests.get(robots_url)

    if response.status_code == 200:
        return response.text
    else:
        return None

def fetch_sitemap(company_url):
    # Append "/sitemap.xml" to the provided URL
    sitemap_url = company_url.rstrip("/") + "/sitemap.xml"

    # Send a GET request to retrieve the content of the sitemap
    response = requests.get(sitemap_url)

    if response.status_code == 200:
        return response.text
    else:
        return None

def extract_sitemap_url(robots_txt_content):
    lines = robots_txt_content.split("\n")
    for line in lines:
        if line.lower().startswith("sitemap:"):
            return line.split(":", 1)[1].strip()

    return None


# Fetch the robots.txt content
robots_txt = fetch_robots_txt(company_url)

if robots_txt:
    # Extract the sitemap URL from the robots.txt content
    sitemap_url = extract_sitemap_url(robots_txt)

    if sitemap_url:
        print(sitemap_url)
    else:
        print("No sitemap URL found in robots.txt")
else:
    print("Failed to fetch robots.txt")



https://www.edapp.com/sitemap.xml


## This filters the urls from the sitemap, pulls the data and stores the html file

In [15]:
import os.path
import datetime
import requests
from bs4 import BeautifulSoup
import datetime
import urllib.request
import ssl
# ...

directory_path = "../Davids_Reports/html_data/"
html_file_name = f"{directory_path}html_output_{company_name}_{timestamp_str}.html"

# Check if a file with similar name already exists
if os.path.exists(html_file_name):
    # Get the modification timestamp of the file
    file_timestamp = datetime.datetime.fromtimestamp(os.path.getmtime(html_file_name))

    # Calculate the difference between the current date and the file's modification date
    time_diff = datetime.datetime.now() - file_timestamp

    # Check if the file was modified within the last XX days and if it belongs to the same company
    if time_diff.days < XX and company_name in html_file_name:
        # Load the existing file content
        with open(html_file_name, "r", encoding="utf-8") as file:
            file_content = file.read()

        # Process the existing file content or perform any other desired actions
        # ...

        print(f"Loaded existing HTML content from '{html_file_name}'.")
    else:
        print(f"File '{html_file_name}' exists but either not updated within the last {XX} days or doesn't belong to the same company. Loading new data.")

        # Load the data from the URLs
        loader = download_loader("BeautifulSoupWebReader")
        documents = loader.load_data(urls=location_urls)

        # Write the documents variable to an HTML file
        with open(html_file_name, "w", encoding="utf-8") as file:
            for doc in documents:
                file.write(str(doc))

        print(f"HTML content saved to '{html_file_name}'.")
else:
    print(f"File '{html_file_name}' does not exist. Loading new data.")

    # Load the data from the URLs
    loader = download_loader("BeautifulSoupWebReader")
    documents = loader.load_data(urls=location_urls)

    # Write the documents variable to an HTML file
    with open(html_file_name, "w", encoding="utf-8") as file:
        for doc in documents:
            file.write(str(doc))

    print(f"HTML content saved to '{html_file_name}'.")

file_path = html_file_name


File '../Davids_Reports/html_data/html_output_Ed App_2024-05-03.html' does not exist. Loading new data.
Failed to load data from URL: https://www.edapp.com/real-rewards/. Error: HTTP Error 404: Not Found
Failed to load data from URL: https://www.edapp.com/canva-authoring-edapp/. Error: HTTP Error 308: Permanent Redirect


KeyboardInterrupt: 

In [ ]:
import os
from llama_index import(
    PromptHelper,
    ServiceContext,
    LLMPredictor
)
from langchain import OpenAI

%cd /content/drive/MyDrive/Colab_Notebooks/Davids_Reports/

current_directory = os.getcwd()
print(current_directory)

from report_modules import llm_report_config

## Now we create an index on the webpage content


IF YOU CHANGE COMPANY ENSURE YOU CHANGE THE CONTENTS FO TEH DATA FOLDER

In [ ]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

if len(documents) > 0:
    print("Documents exist in the list.")
else:
    print("No documents found.")

In [ ]:
index.storage_context.persist()


## This is where we can load our index once generated

ENSURE STORAGE LOCATION IS THE CORRECT ONE FOR THE BUSINESS


In [ ]:
from llama_index import StorageContext, load_index_from_storage

%cd /content/drive/MyDrive/Colab_Notebooks/Davids_Reports/

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage/edume/")
# load index
index = load_index_from_storage(storage_context)

In [ ]:
print(company_name)

In [ ]:
##to reload the most upto date questions remember to delete the # before %load
##also run this section again after load as the code wont have been ran yet


questions = [
        "How does this product benefit front line workers?",
        "What Artificial Intelligence features do they have?",
        "What problems does this product aim to solve?",
        "How does this product reference learning in the flow of work?",
        "How does this product deliver relevant content to its users?",
        "Describe how their product works.",
        "What is their elevator pitch?",
        "How do users interact with the product?",
        "What industries is the product targeting?",
        "What are the key personas this product designed for?",
        "What integrations do they offer?",
        "What's their unique selling proposition",
        "What are their Strengths",
        "What are their weaknesses",
        "What are their opportunities",
        "What are their threats",
        "Summarise their SWOT analysis",
        "What appears to be their Got to Market Strategy",
        "How does their product support automation of admin activities",
        "What does their Developer support provide",
        "How can you embed their product into your app or website",
        "What are the different pricing plans that they offer and whats the differnce between them"
    ]
answer_formats = [
        " provide a summary and bulleted list.",
        " provide a summary and bulleted list.",
        " provide a summary and bulleted list.",
        " provide an indepth explaination.",
        " provide an indepth explaination.",
        " provide an indepth explaination.",
        " This should be a short and concise paragraph, with a 'they are the X for Y' analogy ",
        " provide an indepth explaination.",
        " provide a summary and bulleted list.",
        " provide a summary and bulleted list.",
        " provide a bulleted list. Name of integration: short description of what it does",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide an indepth explaination.",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
        " provide a bulleted list with brief descriptions, and finish with a short summary",
    ]

prompt_guidance = f"do_not_store_history; Do not reference previous answers, always answer the question being asked. You are taking the role of a market analyst reviewing a competitors website. Keep the answers relevant to {company_name} Focus on the training platform product that they produce, not individual courses or materials they provide"
question_and_format = [str(x) + str(y) for x, y in zip(questions, answer_formats)]

question_full = list(map(lambda x: x + prompt_guidance, question_and_format))


##Query the index and docs

In [ ]:

query_engine = index.as_query_engine()
# Generate answers
answers = [query_engine.query(question) for question in question_full]

In [ ]:
# Specify the file name using the timestamp
file_name = f"responses_{company_name}_{timestamp_str}.txt"

# Print the question and answer pairs and write them to the text file
with open(file_name, 'w') as file:
    for question, answer in zip(questions, answers):
        print("Question:", question)
        print("Answer:", answer)
        print()

        file.write("Question: " + question + '\n')
        file.write("Answer: " + str(answer) + '\n')
        file.write('\n')

print(f"Responses saved to {file_name}")

In [ ]:
query_engine = index.as_query_engine()
# Generate answers
additional_question = [query_engine.query(question) for question in question_full]

In [ ]:

%cd /content/drive/MyDrive/Colab_Notebooks/Davids_Reports/report_modules/

import question_prompt_master

%cd /content/drive/MyDrive/Colab_Notebooks/Davids_Reports/

question_full = question_prompt_master.question_full